In [1]:
import sys
sys.path.insert(0, '/home/wdwatson2/projects/CAT-Transformer/model')
# sys.path.insert(0, r'C:\Users\smbm2\projects\CAT-Transformer\model')
# sys.path.insert(0, '/home/warin/projects/CAT-Transformer/model')
from testingModel import CATTransformer, MyFTTransformer, Combined_Dataset, train, test, EarlyStopping
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import optuna
from optuna.trial import TrialState

device_in_use = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device_in_use)

cuda


/home/wdwatson2/miniconda3/envs/ml-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_train = pd.read_csv('/home/wdwatson2/projects/CAT-Transformer/datasets/income/train.csv')
df_test = pd.read_csv('/home/wdwatson2/projects/CAT-Transformer/datasets/income/test.csv')
df_val = pd.read_csv('/home/wdwatson2/projects/CAT-Transformer/datasets/income/validation.csv') 

cont_columns = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week']
cat_columns = ['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country']
cat_features = (10,16,7,16,6,5,2,43)
target = ['income']

#CHECKING TO MAKE SURE YOUR LIST IS CORRECT (NO NEED TO TOUCH)
yourlist = cont_columns + cat_columns+target
yourlist.sort()
oglist = list(df_train.columns)
oglist.sort()

assert(yourlist == oglist), "You may of spelled feature name wrong or you forgot to put on of them in the list"

cat_features = ()

target_classes = [max(len(df_train[target].value_counts()), len(df_val[target].value_counts()),len(df_test[target].value_counts()))]
print(target_classes)
# Create a StandardScaler and fit it to the cont features
scaler = StandardScaler()
scaler.fit(df_train[cont_columns])

# Transform the training, test, and validation datasets
df_train[cont_columns] = scaler.transform(df_train[cont_columns])
df_test[cont_columns] = scaler.transform(df_test[cont_columns])
df_val[cont_columns] = scaler.transform(df_val[cont_columns])

#Wrapping in Dataset
train_dataset = Combined_Dataset(df_train, cat_columns=cat_columns, num_columns=cont_columns, task1_column=target[0])
val_dataset = Combined_Dataset(df_val, cat_columns=cat_columns, num_columns=cont_columns, task1_column=target[0])
test_dataset = Combined_Dataset(df_test, cat_columns=cat_columns, num_columns=cont_columns, task1_column=target[0])

batch_size = 256

# Wrapping with DataLoader for easy batch extraction
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

[2]


In [5]:
def objective(trial):
    trial_number = trial.number

    # Define hyperparameters to search over
    alpha = trial.suggest_float('sigma', 0.001, 5, log=True)
    num_layers = trial.suggest_int('num_layers', 1, 5)
    # Ensure that embed_size is divisible by num_layers
    embed_size = trial.suggest_categorical("embed_size", [50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200, 250, 350, 500])
    heads = trial.suggest_categorical("heads", [1, 5, 10])
    forward_expansion = trial.suggest_int('forward_expansion', 1, 8)
    pre_norm_on = trial.suggest_categorical('prenorm_on', [True, False])
    mlp_scale_classification = trial.suggest_int('mlp_scale_classification', 1, 8)
    decoder_dropout = trial.suggest_categorical('decoder_dropout', [0,.1,.2,.5])
    classification_dropout = trial.suggest_categorical('class_drop', [0,.1,.2,.5])

    learning_rate = trial.suggest_float('learning_rate', 0.00001, 0.001, log=True)
    weight_decay = trial.suggest_float('weight_decay', 0.000001, 0.001, log=True)

    num_epochs = 400

    # Create your model with the sampled hyperparameters
    model = CATTransformer(alpha = alpha,
                           embed_size= embed_size,
                           n_cont = len(cont_columns),
                           cat_feat=cat_features,
                           num_layers=num_layers,
                           heads=heads,
                           forward_expansion=forward_expansion,
                           decoder_dropout=decoder_dropout,
                           classification_dropout=classification_dropout,
                           pre_norm_on=pre_norm_on,
                           mlp_scale_classification=mlp_scale_classification,
                           targets_classes=target_classes,
                           ).to(device_in_use)

    # Define loss function and optimizer
    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay=weight_decay)

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=10)  # Adjust patience as needed

    # Training loop with a large number of epochs
    for epoch in range(num_epochs):
        train_loss, train_acc, train_f1= train(regression_on=False, 
                                  get_attn=False,
                                   dataloader=train_dataloader, 
                                   model=model, 
                                   loss_function=loss_function, 
                                   optimizer=optimizer, 
                                   device_in_use=device_in_use)
        val_loss, val_acc, test_f1 = test(regression_on=False,
                               get_attn=False,
                               dataloader=val_dataloader,
                               model=model,
                               loss_function=loss_function,
                               device_in_use=device_in_use)
        # Check if we should early stop based on validation accuracy
        if early_stopping(val_acc):
            break

    
    # Log the final test accuracy for this trial to a shared log file
    final_log = f"Trial {trial_number} completed. Validation Accuracy = {val_acc:.4f}"

    # Return the test accuracy as the objective to optimize
    return val_acc

In [6]:
# Set the number of optimization trials
num_trials = 150

# Create an Optuna study
study = optuna.create_study(direction='maximize')  # Maximize validation accuracy

# Start the optimization process
study.optimize(objective, n_trials=num_trials, show_progress_bar=True)

# Get the best hyperparameters and the validation accuracy at the point of early stopping
best_params = study.best_params
best_val_accuracy = study.best_value

print("Best Hyperparameters:", best_params)
print("Best Validation Accuracy (at Early Stopping):", best_val_accuracy)

[I 2024-01-06 16:10:26,243] A new study created in memory with name: no-name-77eb95a6-00ec-4067-8b4c-8f4159bd7132
Best trial: 0. Best value: 0.831599:   1%|          | 1/150 [04:34<11:20:37, 274.07s/it]

[I 2024-01-06 16:15:00,317] Trial 0 finished with value: 0.8315987032929534 and parameters: {'sigma': 0.46744698750436525, 'num_layers': 3, 'embed_size': 160, 'heads': 5, 'forward_expansion': 7, 'prenorm_on': False, 'mlp_scale_classification': 4, 'decoder_dropout': 0.1, 'class_drop': 0, 'learning_rate': 2.9917460732420438e-05, 'weight_decay': 5.964995167167225e-05}. Best is trial 0 with value: 0.8315987032929534.


Best trial: 0. Best value: 0.831599:   1%|▏         | 2/150 [09:29<11:47:32, 286.84s/it]

[I 2024-01-06 16:19:56,094] Trial 1 finished with value: 0.8259682648012284 and parameters: {'sigma': 0.046281318975548484, 'num_layers': 3, 'embed_size': 250, 'heads': 5, 'forward_expansion': 5, 'prenorm_on': True, 'mlp_scale_classification': 4, 'decoder_dropout': 0.2, 'class_drop': 0, 'learning_rate': 1.0263869059749554e-05, 'weight_decay': 0.000283847144236642}. Best is trial 0 with value: 0.8315987032929534.


Best trial: 0. Best value: 0.831599:   2%|▏         | 3/150 [12:45<10:01:00, 245.31s/it]

...

[I 2024-01-07 03:25:09,357] Trial 147 finished with value: 0.8377409998293807 and parameters: {'sigma': 4.133380161504655, 'num_layers': 4, 'embed_size': 60, 'heads': 1, 'forward_expansion': 7, 'prenorm_on': False, 'mlp_scale_classification': 2, 'decoder_dropout': 0.2, 'class_drop': 0, 'learning_rate': 5.710637493314308e-05, 'weight_decay': 8.185557969002911e-05}. Best is trial 88 with value: 0.8396178126599556.


Best trial: 88. Best value: 0.839618:  99%|█████████▉| 149/150 [11:18:56<04:06, 246.15s/it]

[I 2024-01-07 03:29:23,080] Trial 148 finished with value: 0.8321105613376557 and parameters: {'sigma': 1.5077718540682061, 'num_layers': 4, 'embed_size': 60, 'heads': 1, 'forward_expansion': 5, 'prenorm_on': False, 'mlp_scale_classification': 2, 'decoder_dropout': 0.2, 'class_drop': 0, 'learning_rate': 5.581621827450652e-05, 'weight_decay': 8.29446490943233e-05}. Best is trial 88 with value: 0.8396178126599556.


Best trial: 88. Best value: 0.839618: 100%|██████████| 150/150 [11:23:12<00:00, 273.28s/it]

[I 2024-01-07 03:33:38,858] Trial 149 finished with value: 0.8298925098106126 and parameters: {'sigma': 2.2423435489310974, 'num_layers': 4, 'embed_size': 60, 'heads': 1, 'forward_expansion': 4, 'prenorm_on': False, 'mlp_scale_classification': 8, 'decoder_dropout': 0.2, 'class_drop': 0, 'learning_rate': 7.396791421711799e-05, 'weight_decay': 6.651081700060878e-05}. Best is trial 88 with value: 0.8396178126599556.
Best Hyperparameters: {'sigma': 1.2680519381758064, 'num_layers': 4, 'embed_size': 500, 'heads': 1, 'forward_expansion': 5, 'prenorm_on': False, 'mlp_scale_classification': 5, 'decoder_dropout': 0.2, 'class_drop': 0.2, 'learning_rate': 0.00010502262647115873, 'weight_decay': 2.134300590129885e-05}
Best Validation Accuracy (at Early Stopping): 0.8396178126599556
